<a href="https://colab.research.google.com/github/kwiju/-_9-_colab/blob/%E5%AE%89%E5%9C%8B%E5%85%89%E7%B4%80/ch07_StatisticalArbitrage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# パッケージのインポート

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.api import VECM

# パラメータの設定

In [ ]:
Sigma = np.array([[1,0,0],[0,1,0],[0,0,1]])

a_1 = np.array([0.1,0.2,0.3]) # alpha_1
a_2 = np.array([0.5,-0.3,-0.3])
b_1 = np.array([1,-0.5,-0.5]) # beta_1
b_2 = np.array([-0.2,0.5,-0.2])

A = np.stack([a_1,a_2],axis = 1) # alpha_2
B = np.stack([b_1,b_2]) # beta_2

Pi_1 = np.dot(a_1.reshape(3,1),b_1.reshape(1,3))
Pi_2 = np.dot(A,B)


In [ ]:
np.random.seed(seed = 1)
rand_nums = np.random.randn(20000,3)

In [ ]:
init = np.array([1,1,1])
S_rw = init
S_ci1 = init
S_ci2 = init

In [ ]:
for rand_num in rand_nums:
    S_rw = np.vstack((S_rw,S_rw[len(S_rw)-1] + np.dot(rand_num,Sigma)))
    S_ci1 = np.vstack((S_ci1,np.dot(Pi_1 + np.eye(3),S_ci1[len(S_ci1)-1])
                       + np.dot(rand_num,Sigma)))
    S_ci2 = np.vstack((S_ci2,np.dot(Pi_2 + np.eye(3),S_ci2[len(S_ci2)-1])
                       + np.dot(rand_num,Sigma)))



# Johansen検定

## $\mathbf{S}_{\textbf{RW}}$をテストする

In [ ]:
JohansenTestResult_rw = coint_johansen(S_rw,k_ar_diff=0,det_order=-1)

In [ ]:
print(JohansenTestResult_rw.lr1) # Trace statistic

[22.58036266  8.78820583  2.43645402]


In [ ]:
print(JohansenTestResult_rw.cvt) # Critical values (90%,95%,99%) of trace statistic

[[21.7781 24.2761 29.5147]
 [10.4741 12.3212 16.364 ]
 [ 2.9762  4.1296  6.9406]]


In [ ]:
print(JohansenTestResult_rw.lr2) # Maximum eigenvalue statistic

[13.79215683  6.35175181  2.43645402]


In [ ]:
print(JohansenTestResult_rw.cvm) # Critical values (90%,95%,99%) of maximum eigenvalue statistic

[[15.7175 17.7961 22.2519]
 [ 9.4748 11.2246 15.0923]
 [ 2.9762  4.1296  6.9406]]


## $\mathbf{S}_{\textbf{CI},1}$をテストする

In [ ]:
JohansenTestResult_s1 = coint_johansen(S_ci1,k_ar_diff=0,det_order=-1)

In [ ]:
print(JohansenTestResult_s1.lr1) # Trace statistic
print(JohansenTestResult_s1.cvt) # Critical values (90%,95%,99%) of trace statistic
print(JohansenTestResult_s1.lr2) # Maximum eigenvalue statistic
print(JohansenTestResult_s1.cvm) # Critical values (90%,95%,99%) of maximum eigenvalue statistic

[1.15097289e+04 1.42336157e+01 1.08408520e+00]
[[21.7781 24.2761 29.5147]
 [10.4741 12.3212 16.364 ]
 [ 2.9762  4.1296  6.9406]]
[1.14954952e+04 1.31495305e+01 1.08408520e+00]
[[15.7175 17.7961 22.2519]
 [ 9.4748 11.2246 15.0923]
 [ 2.9762  4.1296  6.9406]]


## $\mathbf{S}_{\textbf{CI,2}}$をテストする

In [ ]:
JohansenTestResult_s2 = coint_johansen(S_ci2,k_ar_diff=0,det_order=-1)

In [ ]:
print(JohansenTestResult_s2.lr1) # Trace statistic
print(JohansenTestResult_s2.cvt) # Critical values (90%,95%,99%) of trace statistic
print(JohansenTestResult_s2.lr2) # Maximum eigenvalue statistic
print(JohansenTestResult_s2.cvm) # Critical values (90%,95%,99%) of maximum eigenvalue statistic

[3.24178966e+04 6.11019131e+03 2.57568342e+00]
[[21.7781 24.2761 29.5147]
 [10.4741 12.3212 16.364 ]
 [ 2.9762  4.1296  6.9406]]
[2.63077053e+04 6.10761563e+03 2.57568342e+00]
[[15.7175 17.7961 22.2519]
 [ 9.4748 11.2246 15.0923]
 [ 2.9762  4.1296  6.9406]]


# モデル・パラメータの推定

## $\mathbf{S}_{\textbf{CI},1}$のパラメータ推定

In [ ]:
model_s1 = VECM(S_ci1,k_ar_diff=0,coint_rank = 1, deterministic='na')
res_s1 = model_s1.fit()


In [ ]:
print(res_s1.summary())

                 Loading coefficients (alpha) for equation y1                 
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ec1            0.0998      0.003     32.514      0.000       0.094       0.106
                 Loading coefficients (alpha) for equation y2                 
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ec1            0.2077      0.003     68.038      0.000       0.202       0.214
                 Loading coefficients (alpha) for equation y3                 
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ec1            0.3019      0.003     98.418      0.000       0.296       0.308
          Cointegration relations for loading-coeffi

In [ ]:
print(res_s1.alpha)
print(res_s1.beta)

[[0.09983864]
 [0.20767309]
 [0.30188794]]
[[ 1.        ]
 [-0.50000816]
 [-0.49972569]]


## $\mathbf{S}_{\textbf{CI},2}$のパラメータ


*   リスト項目
*   リスト項目



In [ ]:
model_s2 = VECM(S_ci2,k_ar_diff=0,coint_rank = 2, deterministic='na')
res_s2 = model_s2.fit()

print(res_s2.alpha)
print(res_s2.beta)

[[ 0.00295261  0.19832263]
 [ 0.26430882 -0.25658122]
 [ 0.36024384 -0.29875471]]
[[ 1.00000000e+00 -1.10297307e-16]
 [ 4.27951128e-17  1.00000000e+00]
 [-8.74973589e-01 -7.49993120e-01]]


In [ ]:
print(np.dot(res_s2.alpha,res_s2.beta.transpose()))
print(Pi_2)


[[ 0.00295261  0.19832263 -0.15132406]
 [ 0.26430882 -0.25658122 -0.03882909]
 [ 0.36024384 -0.29875471 -0.09113987]]
[[ 0.    0.2  -0.15]
 [ 0.26 -0.25 -0.04]
 [ 0.36 -0.3  -0.09]]


In [ ]:
kouki